In [4]:
import geopandas as gpd
import networkx as nx
import fiona
import tqdm
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import glob

from shapely.ops import cascaded_union
from shapely.wkt import loads 
from shapely.geometry import Point
from scipy.spatial import cKDTree

# 使用データ

In [2]:
"""
'Lakshminiya'データないエラー
"""

"\n'Lakshminiya'データないエラー\n"

In [3]:
district = gpd.read_file(r'C:\Users\kurokiso\Desktop\道路ネットワーク\取得情報\hermes_NPL_new_wgs\hermes_NPL_new_wgs_3.shp')
use_dis_list = ['Bhadrapur', 'Dharan', 'Biratnagar', 'Rajbiraj', 'Lahan', 'Janakpur', 'Lakshminiya', 'Jaleswor', 'Gaushala', 'Malangawa', 'Gaur', 'Hetauda', 'Kalaiya', 'Birgunj', 'Bharatpur', 'Pokhara Lekhnath', 'Butwal', 'Siddharthanagar', 'Kapilbastu', 'Krishnanagar', 'Nepalgunj', 'Dhangadhi', 'Bhimdatta', ]
water = gpd.read_file(r'C:\Users\kurokiso\Desktop\道路ネットワーク\water_gpkg\all.gpkg')
source = gpd.read_file(r'C:\Users\kurokiso\Desktop\道路ネットワーク\water_gpkg\sou_E.gpkg')
road = gpd.read_file(r'C:\Users\kurokiso\Desktop\道路ネットワーク\npl-rdsl-trans-25k-50k-sdn-wgs84\npl_rdsl_trans_25K_50K_sdn_wgs84.shp')
road = road.reset_index()

In [4]:
def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

# overlap

### カトマンズ以外の都市

In [65]:
use_dis_list = ['Bhadrapur', 'Bharatpur', 'Bhimdatta', 'Biratnagar', 'Birgunj', 'Butwal', 'Dhangadhi', 'Dharan', 'Gaur', 'Gaushala', 'Hetauda', 'Jaleswor', 'Janakpur', 'Kalaiya', 'Kapilbastu', 'Krishnanagar', 'Lahan', 'Malangawa', 'Nepalgunj', 'Pokhara Lekhnath', 'Rajbiraj', 'Siddharthanagar']

In [67]:
for i in use_dis_list:
    local = i
    os.mkdir(local)
    # 対象地区の取り出し
    df_dis = district[district['LOCAL'] == i]
    water_clipped = gpd.clip(water, df_dis)
    source_clipped = gpd.clip(source, df_dis)
    road_clipped = gpd.clip(road, df_dis)
    gpd.GeoDataFrame.to_file(water_clipped, "{}/water.gpkg".format(local), driver="GPKG", encoding='utf-8')
    gpd.GeoDataFrame.to_file(source_clipped, "{}/source.gpkg".format(local), driver="GPKG", encoding='utf-8')
    gpd.GeoDataFrame.to_file(road_clipped[['index','LENGTH', 'geometry']], "{}/road.gpkg".format(local), driver="GPKG", encoding='utf-8')    
    
    # 始点と終点の抜き出し
    id_list = []
    index_list = []
    geopoint_list = []
    start_id = 0
    with fiona.open('{}/road.gpkg'.format(local)) as lines:
        for line in lines:
            geopoint_list.append(str(Point(line['geometry']['coordinates'][0])))
            id_list.append(start_id)
            start_id += 1
            index_list.append(line['properties']['index'])
            geopoint_list.append(str(Point(line['geometry']['coordinates'][-1])))
            id_list.append(start_id)
            start_id += 1
            index_list.append(line['properties']['index'])
    df = pd.DataFrame({'id': id_list, 'index': index_list, 'geometry': geopoint_list})
    df = gpd.GeoDataFrame(df, geometry=[loads(wkt) for wkt in df['geometry']])
    df.to_file('{}/vertices.gpkg'.format(local), driver='GPKG')
    
    # 重なっている点を取り出す
    df['X'] = df['geometry'].x
    df['Y'] = df['geometry'].y
    df2 = pd.DataFrame(df.groupby(['X', 'Y']).agg(id=('id', lambda x: list(x.value_counts().index))))
    dup_list = df2['id'].values
    
    # uniqeu点のみ取り出す
    unique_list = []
    for i in dup_list:
        unique_list.append(i[0])
    unique_df = df.set_index('id').iloc[unique_list, :].reset_index()[['id', 'index', 'geometry']]
    unique_df.to_file('{}/uqnique_vertices.gpkg'.format(local), driver='GPKG')

    # 2点と距離の情報
    start_id_list = []
    end_id_list = []
    length_list = []
    index_list = []
    start_id = 0
    with fiona.open('{}/road.gpkg'.format(local)) as lines:
        for line in lines:
            start_id_list.append(start_id)
            start_id += 1
            end_id_list.append(start_id)
            start_id += 1
            length_list.append(line['properties']['LENGTH'])  
            index_list.append(line['properties']['index'])
    distance = pd.DataFrame({'start_id': start_id_list, 'end_id': end_id_list, 'length': length_list, 'index': index_list})
    
    # uniqueの点に統一する
    dic = {}
    for i in dup_list:
        for v in i:
            dic[v] = i[0]
    distance['start_id'] = distance['start_id'].apply(lambda x: dic[x])
    distance['end_id'] = distance['end_id'].apply(lambda x: dic[x])
    
    # 隣接行列作成
    node_num = len(id_list)
    INF = 10**5 - 1
    W = [[INF] * node_num for _ in range(node_num)]
    for i in range(distance.shape[0]):
        W[distance.loc[i, 'start_id']][distance.loc[i, 'end_id']] = distance.loc[1, 'length']
        W[distance.loc[i, 'end_id']][distance.loc[i, 'start_id']] = distance.loc[1, 'length']
    
    # 井戸最近棒
    source_nearest = ckdnearest(source_clipped, unique_df)[['Code', 'id']]
    
    # 水道の最近棒
    water_nearest = ckdnearest(water_clipped, unique_df)[['Rowid', 'id']]
    
    # 経路まとめ
    node_num = len(id_list)
    INF = 0
    W = [[INF] * node_num for _ in range(node_num)]
    for i in range(distance.shape[0]):
        W[distance.loc[i, 'start_id']][distance.loc[i, 'end_id']] = distance.loc[i, 'length']
        W[distance.loc[i, 'end_id']][distance.loc[i, 'start_id']] = distance.loc[i, 'length']

    data = np.array(W)
    G=nx.from_numpy_matrix(data)

    water_rowid_list = []
    nearest_dist_list = []
    nearest_unite_list = []
    for i in range(water_nearest.shape[0]):
        water_id = water_nearest.loc[i, 'id']
        dist_list = []
        unite_list = []
        for v in range(source_nearest.shape[0]):
            source_id = source_nearest.loc[v, 'id']
            try:
                nearest_point = nx.shortest_path(G, source=water_id, target=source_id, weight='weight')
                nearest_dis = nx.shortest_path_length(G, source=water_id, target=source_id, weight='weight')
                dist_list.append(nearest_dis)
                unite_list.append(nearest_point)
            except:
                continue
        try:
            min_dist_index = dist_list.index(min(dist_list))
            min_dist = dist_list[min_dist_index]
            min_unite = unite_list[min_dist_index]
            nearest_dist_list.append(min_dist)
            nearest_unite_list.append(min_unite)
            water_rowid_list.append(water_nearest.loc[i, 'Rowid'])
        except:
            continue
    output = pd.DataFrame({'Rowid': water_rowid_list, 'nearest_source_dist': nearest_dist_list, 'nearest_unite_list': nearest_unite_list})
    output.to_csv('{}/output.csv'.format(local))
    
    set_list = []
    for i in range(distance.shape[0]):
        b = {distance.loc[i, 'start_id'], distance.loc[i, 'end_id']}
        set_list.append(b)
    # おまけ unite 点、道路情報の出力
    for i in range(output.shape[0]):
        point_list = output.loc[i, 'nearest_unite_list']
        rowid = output.loc[i, 'Rowid']
        nearest_point_info = unique_df.set_index('id').loc[point_list, :].reset_index()[['id', 'index', 'geometry']]
        nearest_point_info.to_file('{}/unite_point{}.gpkg'.format(local, rowid), driver='GPKG')

        point_list = output.loc[i, 'nearest_unite_list']
        rowid = output.loc[i, 'Rowid']
        index_list = []
        for i in range(len(point_list)-1):
            a = {point_list[i], point_list[i+1]}

            for i in range(len(set_list)):
                if set_list[i] == a:
                    index_list.append(i)

        road_index_list = distance.loc[index_list, 'index']
        road_info = road[road['index'].isin(road_index_list)][['index', 'LENGTH', 'geometry']]

        try:
            road_info.to_file('{}/road_info{}.gpkg'.format(local, rowid), driver='GPKG')
        except:
            nearest_point_info.to_file('{}/road_info{}.gpkg'.format(local, rowid), driver='GPKG')

C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()
C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to g

C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()
C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to g

# カトマンズ

In [57]:
df_dis_list = ['Tarakeshwor', 'Tokha', 'Budhanilakantha', 'Gokarneshwor', 'Kageshwori Manahora', 'Changunarayan', 'Madhyapur Thimi', 'Bhaktapur', 'Kathmandu', 'Kirtipur', 'Lalitpur', 'Dakshinkali', 'Godawari']

In [6]:
df_dis = district[district['LOCAL'].isin(df_dis_list)]
df_dis = gpd.GeoSeries(cascaded_union(df_dis.geometry))

local = 'kathmandu'
os.mkdir(local)
water_clipped = gpd.clip(water, df_dis)
source_clipped = gpd.clip(source, df_dis)
road_clipped = gpd.clip(road, df_dis)
gpd.GeoDataFrame.to_file(water_clipped, "{}/water.gpkg".format(local), driver="GPKG", encoding='utf-8')
gpd.GeoDataFrame.to_file(source_clipped, "{}/source.gpkg".format(local), driver="GPKG", encoding='utf-8')
gpd.GeoDataFrame.to_file(road_clipped[['index','LENGTH', 'geometry']], "{}/road.gpkg".format(local), driver="GPKG", encoding='utf-8')    

C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()


In [60]:
df_dis = district[district['LOCAL'].isin(df_dis_list)]
df_dis = gpd.GeoSeries(cascaded_union(df_dis.geometry))

local = 'kathmandu'
os.mkdir(local)
water_clipped = gpd.clip(water, df_dis)
source_clipped = gpd.clip(source, df_dis)
road_clipped = gpd.clip(road, df_dis)
gpd.GeoDataFrame.to_file(water_clipped, "{}/water.gpkg".format(local), driver="GPKG", encoding='utf-8')
gpd.GeoDataFrame.to_file(source_clipped, "{}/source.gpkg".format(local), driver="GPKG", encoding='utf-8')
gpd.GeoDataFrame.to_file(road_clipped[['index','LENGTH', 'geometry']], "{}/road.gpkg".format(local), driver="GPKG", encoding='utf-8')    

# 始点と終点の抜き出し
id_list = []
index_list = []
geopoint_list = []
start_id = 0
with fiona.open('{}/road.gpkg'.format(local)) as lines:
    for line in lines:
        geopoint_list.append(str(Point(line['geometry']['coordinates'][0])))
        id_list.append(start_id)
        start_id += 1
        index_list.append(line['properties']['index'])
        geopoint_list.append(str(Point(line['geometry']['coordinates'][-1])))
        id_list.append(start_id)
        start_id += 1
        index_list.append(line['properties']['index'])
df = pd.DataFrame({'id': id_list, 'index': index_list, 'geometry': geopoint_list})
df = gpd.GeoDataFrame(df, geometry=[loads(wkt) for wkt in df['geometry']])
df.to_file('{}/vertices.gpkg'.format(local), driver='GPKG')
print(df)

# 重なっている点を取り出す
df['X'] = df['geometry'].x
df['Y'] = df['geometry'].y
df2 = pd.DataFrame(df.groupby(['X', 'Y']).agg(id=('id', lambda x: list(x.value_counts().index))))
dup_list = df2['id'].values
print(df2)

# uniqeu点のみ取り出す
unique_list = []
for i in dup_list:
    unique_list.append(i[0])
unique_df = df.set_index('id').iloc[unique_list, :].reset_index()[['id', 'index', 'geometry']]
unique_df.to_file('{}/uqnique_vertices.gpkg'.format(local), driver='GPKG')
print(unique_df)

# 2点と距離の情報
start_id_list = []
end_id_list = []
length_list = []
index_list = []
start_id = 0
with fiona.open('{}/road.gpkg'.format(local)) as lines:
    for line in lines:
        start_id_list.append(start_id)
        start_id += 1
        end_id_list.append(start_id)
        start_id += 1
        length_list.append(line['properties']['LENGTH'])  
        index_list.append(line['properties']['index'])
distance = pd.DataFrame({'start_id': start_id_list, 'end_id': end_id_list, 'length': length_list, 'index': index_list})

# uniqueの点に統一する
dic = {}
for i in dup_list:
    for v in i:
        dic[v] = i[0]
distance['start_id'] = distance['start_id'].apply(lambda x: dic[x])
distance['end_id'] = distance['end_id'].apply(lambda x: dic[x])

# 隣接行列作成
node_num = len(id_list)
INF = 10**5 - 1
W = [[INF] * node_num for _ in range(node_num)]
for i in range(distance.shape[0]):
    W[distance.loc[i, 'start_id']][distance.loc[i, 'end_id']] = distance.loc[1, 'length']
    W[distance.loc[i, 'end_id']][distance.loc[i, 'start_id']] = distance.loc[1, 'length']

# 井戸最近棒
source_nearest = ckdnearest(source_clipped, unique_df)[['Code', 'id']]

# 水道の最近棒
water_nearest = ckdnearest(water_clipped, unique_df)[['Rowid', 'id']]

# 経路まとめ
node_num = len(id_list)
INF = 0
W = [[INF] * node_num for _ in range(node_num)]
for i in range(distance.shape[0]):
    W[distance.loc[i, 'start_id']][distance.loc[i, 'end_id']] = distance.loc[i, 'length']
    W[distance.loc[i, 'end_id']][distance.loc[i, 'start_id']] = distance.loc[i, 'length']

data = np.array(W)
G=nx.from_numpy_matrix(data)

water_rowid_list = []
nearest_dist_list = []
nearest_unite_list = []
for i in range(water_nearest.shape[0]):
    water_id = water_nearest.loc[i, 'id']
    dist_list = []
    unite_list = []
    for v in range(source_nearest.shape[0]):
        source_id = source_nearest.loc[v, 'id']
        try:
            nearest_point = nx.shortest_path(G, source=water_id, target=source_id, weight='weight')
            nearest_dis = nx.shortest_path_length(G, source=water_id, target=source_id, weight='weight')
            dist_list.append(nearest_dis)
            unite_list.append(nearest_point)
        except:
            continue
    try:
        min_dist_index = dist_list.index(min(dist_list))
        min_dist = dist_list[min_dist_index]
        min_unite = unite_list[min_dist_index]
        nearest_dist_list.append(min_dist)
        nearest_unite_list.append(min_unite)
        water_rowid_list.append(water_nearest.loc[i, 'Rowid'])
        print('-----------------------------------------------------------------------------------------------')
        print('water', water_id)
        print(min_dist)
        print(min_unite)
    except:
        continue
output = pd.DataFrame({'Rowid': water_rowid_list, 'nearest_source_dist': nearest_dist_list, 'nearest_unite_list': nearest_unite_list})
output.to_csv('{}/output.csv'.format(local))


set_list = []
for i in range(distance.shape[0]):
    b = {distance.loc[i, 'start_id'], distance.loc[i, 'end_id']}
    set_list.append(b)
# おまけ unite 点、道路情報の出力
for i in range(output.shape[0]):
    point_list = output.loc[i, 'nearest_unite_list']
    rowid = output.loc[i, 'Rowid']
    nearest_point_info = unique_df.set_index('id').loc[point_list, :].reset_index()[['id', 'index', 'geometry']]
    nearest_point_info.to_file('{}/unite_point{}.gpkg'.format(local, rowid), driver='GPKG')
    
    point_list = output.loc[i, 'nearest_unite_list']
    rowid = output.loc[i, 'Rowid']
    index_list = []
    for i in range(len(point_list)-1):
        a = {point_list[i], point_list[i+1]}

        for i in range(len(set_list)):
            if set_list[i] == a:
                index_list.append(i)

    road_index_list = distance.loc[index_list, 'index']
    road_info = road[road['index'].isin(road_index_list)][['index', 'LENGTH', 'geometry']]
    
    try:
        road_info.to_file('{}/road_info{}.gpkg'.format(local, rowid), driver='GPKG')
    except:
        nearest_point_info.to_file('{}/road_info{}.gpkg'.format(local, rowid), driver='GPKG')

-----------------------------------------------------------------------------------------------
water 17495
2862.679
[17495, 17547, 17559, 17911, 17835, 17739, 17865, 20981, 17959, 17958, 17887, 17862]
-----------------------------------------------------------------------------------------------
water 17911
1842.5974
[17911, 17835, 17739, 17865, 20981, 17959, 17958, 17887, 17862]
-----------------------------------------------------------------------------------------------
water 17057
2375.72492
[17057, 16975, 16979, 17030, 17187, 17186, 20828, 17395, 17427, 17426, 20901, 17883, 17891, 17862]
-----------------------------------------------------------------------------------------------
water 17254
2007.63759
[17254, 20829, 20828, 17395, 17427, 17426, 20901, 17883, 17891, 17862]
-----------------------------------------------------------------------------------------------
water 21061
418.03299
[21061, 17879, 17893, 17891, 17862]
------------------------------------------------------

-----------------------------------------------------------------------------------------------
water 20323
978.444695
[20323, 20322, 20316, 20274, 20275, 15111, 15082]
-----------------------------------------------------------------------------------------------
water 17307
3475.89854
[17307, 17229, 17270, 17142, 17143, 20763, 20767, 16770, 16667, 20709, 16476, 16449, 20641, 16463, 16347, 16010, 15700, 20410, 15528, 15250, 15166, 15167, 15082]
-----------------------------------------------------------------------------------------------
water 7799
0
[7799]
-----------------------------------------------------------------------------------------------
water 15351
1288.5609949999998
[15351, 20237, 20236, 20275, 15111, 15082]
-----------------------------------------------------------------------------------------------
water 15351
1288.5609949999998
[15351, 20237, 20236, 20275, 15111, 15082]
----------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------
water 4974
5548.95379
[4974, 8995, 9043, 9042, 9715, 9183, 9223, 9222, 9731, 9743, 9427, 9744, 9770, 9543, 9579, 9603]
-----------------------------------------------------------------------------------------------
water 11367
2478.771843
[11367, 11371, 11391, 11479, 12423, 12422, 11687, 11768, 17438, 17439, 17478, 17423, 17410, 17411, 17402, 17382, 17383, 17379, 17375]
-----------------------------------------------------------------------------------------------
water 11367
2478.771843
[11367, 11371, 11391, 11479, 12423, 12422, 11687, 11768, 17438, 17439, 17478, 17423, 17410, 17411, 17402, 17382, 17383, 17379, 17375]
-----------------------------------------------------------------------------------------------
water 10039
3136.3722399999997
[10039, 10027, 10035, 10049, 10063, 10062, 10083, 10082, 13597, 13570, 13626, 13654, 13662, 13719, 19686, 19687, 13874, 13875, 19735, 14054, 14055, 14

# notoverlap

### 手作業後（カトマンズ以外）

In [5]:
use_dis_list = ['Bhadrapur', 'Bharatpur', 'Bhimdatta', 'Biratnagar', 'Birgunj', 'Butwal', 'Dhangadhi', 'Dharan', 'Gaur', 'Gaushala', 'Hetauda', 'Jaleswor', 'Janakpur', 'Kalaiya', 'Kapilbastu', 'Krishnanagar', 'Lahan', 'Malangawa', 'Nepalgunj', 'Pokhara Lekhnath', 'Rajbiraj', 'Siddharthanagar']

In [26]:
for i in use_dis_list:
    local = i
    os.mkdir(local)
    # 対象地区の取り出し
    df_dis = district[district['LOCAL'] == i]
    water_clipped = gpd.clip(water, df_dis)
    source_clipped = gpd.clip(source, df_dis)
    road_clipped = gpd.clip(road, df_dis)
    gpd.GeoDataFrame.to_file(water_clipped, "{}/water.gpkg".format(local), driver="GPKG", encoding='utf-8')
    gpd.GeoDataFrame.to_file(source_clipped, "{}/source.gpkg".format(local), driver="GPKG", encoding='utf-8')
    gpd.GeoDataFrame.to_file(road_clipped[['index','LENGTH', 'geometry']], "{}/road.gpkg".format(local), driver="GPKG", encoding='utf-8')    
    
    # 始点と終点の抜き出し
    id_list = []
    index_list = []
    geopoint_list = []
    start_id = 0
    with fiona.open('{}/road.gpkg'.format(local)) as lines:
        for line in lines:
            geopoint_list.append(str(Point(line['geometry']['coordinates'][0])))
            id_list.append(start_id)
            start_id += 1
            index_list.append(line['properties']['index'])
            geopoint_list.append(str(Point(line['geometry']['coordinates'][-1])))
            id_list.append(start_id)
            start_id += 1
            index_list.append(line['properties']['index'])
    df = pd.DataFrame({'id': id_list, 'index': index_list, 'geometry': geopoint_list})
    df = gpd.GeoDataFrame(df, geometry=[loads(wkt) for wkt in df['geometry']])
    
    # 重なっている点を取り出す
    df['X'] = df['geometry'].x
    df['Y'] = df['geometry'].y
    df2 = pd.DataFrame(df.groupby(['X', 'Y']).agg(id=('id', lambda x: list(x.value_counts().index))))
    dup_list = df2['id'].values
    
    # uniqeu点のみ取り出す
    unique_list = []
    for i in dup_list:
        unique_list.append(i[0])
    unique_df = df.set_index('id').iloc[unique_list, :].reset_index()[['id', 'index', 'geometry']]
    unique_df.to_file('{}/uqnique_vertices.gpkg'.format(local), driver='GPKG')

    # 2点と距離の情報
    start_id_list = []
    end_id_list = []
    length_list = []
    index_list = []
    start_id = 0
    with fiona.open('{}/road.gpkg'.format(local)) as lines:
        for line in lines:
            start_id_list.append(start_id)
            start_id += 1
            end_id_list.append(start_id)
            start_id += 1
            length_list.append(line['properties']['LENGTH'])  
            index_list.append(line['properties']['index'])
    distance = pd.DataFrame({'start_id': start_id_list, 'end_id': end_id_list, 'length': length_list, 'index': index_list})
    
    # uniqueの点に統一する
    dic = {}
    for i in dup_list:
        for v in i:
            dic[v] = i[0]
    distance['start_id'] = distance['start_id'].apply(lambda x: dic[x])
    distance['end_id'] = distance['end_id'].apply(lambda x: dic[x])
    
    # 重なりのない経路抽出
    output = pd.read_csv(r'C:\Users\kurokiso\Desktop\道路ネットワーク\result\right_left\{}\output_flow2.csv'.format(local))
    set_list = []
    for i in range(distance.shape[0]):
        b = {distance.loc[i, 'start_id'], distance.loc[i, 'end_id']}
        set_list.append(b)
    # unite 点、道路情報の出力
    for i in range(output.shape[0]):
        point_list = eval(output.loc[i, 'nearest_unite_list'])
        rowid = output.loc[i, 'Rowid']
        nearest_point_info = unique_df.set_index('id').loc[point_list, :].reset_index()[['id', 'index', 'geometry']]
        nearest_point_info.to_file('{}/unite_point{}.gpkg'.format(local, rowid), driver='GPKG')

        point_list = eval(output.loc[i, 'nearest_unite_list'])
        rowid = output.loc[i, 'Rowid']
        index_list = []
        for i in range(len(point_list)-1):
            a = {point_list[i], point_list[i+1]}

            for i in range(len(set_list)):
                if set_list[i] == a:
                    index_list.append(i)

        road_index_list = distance.loc[index_list, 'index']
        road_info = road[road['index'].isin(road_index_list)][['index', 'LENGTH', 'geometry']]

        try:
            road_info.to_file('{}/road_info{}.gpkg'.format(local, rowid), driver='GPKG')
        except:
            nearest_point_info.to_file('{}/road_info{}.gpkg'.format(local, rowid), driver='GPKG')

C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()
C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to g

C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()
C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to g

# 手作業後（カトマンズ）

In [27]:
df_dis_list = ['Tarakeshwor', 'Tokha', 'Budhanilakantha', 'Gokarneshwor', 'Kageshwori Manahora', 'Changunarayan', 'Madhyapur Thimi', 'Bhaktapur', 'Kathmandu', 'Kirtipur', 'Lalitpur', 'Dakshinkali', 'Godawari']

In [28]:
df_dis = district[district['LOCAL'].isin(df_dis_list)]
df_dis = gpd.GeoSeries(cascaded_union(df_dis.geometry))

local = 'kathmandu'
os.mkdir(local)
water_clipped = gpd.clip(water, df_dis)
source_clipped = gpd.clip(source, df_dis)
road_clipped = gpd.clip(road, df_dis)
gpd.GeoDataFrame.to_file(water_clipped, "{}/water.gpkg".format(local), driver="GPKG", encoding='utf-8')
gpd.GeoDataFrame.to_file(source_clipped, "{}/source.gpkg".format(local), driver="GPKG", encoding='utf-8')
gpd.GeoDataFrame.to_file(road_clipped[['index','LENGTH', 'geometry']], "{}/road.gpkg".format(local), driver="GPKG", encoding='utf-8')    

C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()


In [30]:
# 始点と終点の抜き出し
id_list = []
index_list = []
geopoint_list = []
start_id = 0
with fiona.open('{}/road.gpkg'.format(local)) as lines:
    for line in lines:
        geopoint_list.append(str(Point(line['geometry']['coordinates'][0])))
        id_list.append(start_id)
        start_id += 1
        index_list.append(line['properties']['index'])
        geopoint_list.append(str(Point(line['geometry']['coordinates'][-1])))
        id_list.append(start_id)
        start_id += 1
        index_list.append(line['properties']['index'])
df = pd.DataFrame({'id': id_list, 'index': index_list, 'geometry': geopoint_list})
df = gpd.GeoDataFrame(df, geometry=[loads(wkt) for wkt in df['geometry']])
df.to_file('{}/vertices.gpkg'.format(local), driver='GPKG')
print(df)

# 重なっている点を取り出す
df['X'] = df['geometry'].x
df['Y'] = df['geometry'].y
df2 = pd.DataFrame(df.groupby(['X', 'Y']).agg(id=('id', lambda x: list(x.value_counts().index))))
dup_list = df2['id'].values
print(df2)

# uniqeu点のみ取り出す
unique_list = []
for i in dup_list:
    unique_list.append(i[0])
unique_df = df.set_index('id').iloc[unique_list, :].reset_index()[['id', 'index', 'geometry']]
unique_df.to_file('{}/uqnique_vertices.gpkg'.format(local), driver='GPKG')
print(unique_df)

# 2点と距離の情報
start_id_list = []
end_id_list = []
length_list = []
index_list = []
start_id = 0
with fiona.open('{}/road.gpkg'.format(local)) as lines:
    for line in lines:
        start_id_list.append(start_id)
        start_id += 1
        end_id_list.append(start_id)
        start_id += 1
        length_list.append(line['properties']['LENGTH'])  
        index_list.append(line['properties']['index'])
distance = pd.DataFrame({'start_id': start_id_list, 'end_id': end_id_list, 'length': length_list, 'index': index_list})

# uniqueの点に統一する
dic = {}
for i in dup_list:
    for v in i:
        dic[v] = i[0]
distance['start_id'] = distance['start_id'].apply(lambda x: dic[x])
distance['end_id'] = distance['end_id'].apply(lambda x: dic[x])

# 重なりのない経路抽出
output = pd.read_csv(r'C:\Users\kurokiso\Desktop\道路ネットワーク\result\right_left\{}\output_flow2.csv'.format(local))
set_list = []
for i in range(distance.shape[0]):
    b = {distance.loc[i, 'start_id'], distance.loc[i, 'end_id']}
    set_list.append(b)
# おまけ unite 点、道路情報の出力
for i in range(output.shape[0]):
    point_list = eval(output.loc[i, 'nearest_unite_list'])
    rowid = output.loc[i, 'Rowid']
    nearest_point_info = unique_df.set_index('id').loc[point_list, :].reset_index()[['id', 'index', 'geometry']]
    nearest_point_info.to_file('{}/unite_point{}.gpkg'.format(local, rowid), driver='GPKG')
    
    point_list = eval(output.loc[i, 'nearest_unite_list'])
    rowid = output.loc[i, 'Rowid']
    index_list = []
    for i in range(len(point_list)-1):
        a = {point_list[i], point_list[i+1]}

        for i in range(len(set_list)):
            if set_list[i] == a:
                index_list.append(i)

    road_index_list = distance.loc[index_list, 'index']
    road_info = road[road['index'].isin(road_index_list)][['index', 'LENGTH', 'geometry']]
    
    try:
        road_info.to_file('{}/road_info{}.gpkg'.format(local, rowid), driver='GPKG')
    except:
        nearest_point_info.to_file('{}/road_info{}.gpkg'.format(local, rowid), driver='GPKG')

          id   index                   geometry
0          0  330684  POINT (80.66917 28.87606)
1          1  330684  POINT (80.66432 28.86933)
2          2  330685  POINT (80.66432 28.86933)
3          3  330685  POINT (80.66223 28.86509)
4          4  330686  POINT (80.66223 28.86509)
...      ...     ...                        ...
21207  21207  639608  POINT (85.35267 27.69928)
21208  21208  639609  POINT (85.30530 27.68052)
21209  21209  639609  POINT (85.30219 27.68114)
21210  21210  639610  POINT (85.32723 27.66334)
21211  21211  639610  POINT (85.32412 27.65980)

[21212 rows x 3 columns]
                         id
X         Y                
80.478636 28.953033  [4049]
80.481944 28.937139  [4051]
80.484108 28.966054  [4043]
80.493402 28.970785  [4042]
80.494926 28.923963  [4052]
...                     ...
85.523712 27.696539  [4188]
85.523934 27.700827  [4185]
85.524078 27.702236  [4180]
85.525334 27.726910  [4054]
85.525997 27.723065  [4059]

[8240 rows x 1 columns]
        i